In [1]:
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
tiff_path = r'C:\Doutorado\3_Trimestre\Analise_Espacial\Enner_Milton\OLI_8\Mesclar.tif'

ds = gdal.Open(tiff_path)

ds.GetMetadata()

{'AREA_OR_POINT': 'Area'}

In [3]:
ds.GetGeoTransform()

(563085.0, 30.0, 0.0, -2280285.0, 0.0, -30.0)

In [4]:
ds.GetProjection()

'PROJCS["WGS 84 / UTM zone 22N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-51],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32622"]]'

In [5]:
def get_GeoTransform_params(ds):
    
    originX, pixelWidth,  a, originY, b, pixelHeight = ds.GetGeoTransform()
    
    
    return originX, pixelWidth, originY, pixelHeight

originX, pixelWidth, originY,  pixelHeight = get_GeoTransform_params(ds)

print(originX, pixelWidth,  originY, pixelHeight)

563085.0 30.0 -2280285.0 -30.0


In [87]:
def get_dx_dy(ds):

    dx = ds.GetGeoTransform()[1]

    dy = ds.GetGeoTransform()[-1]
    return dx, dy

dx, dy = get_dx_dy(ds)

print(dx, dy)

def get_XSize_YSize(ds, band):
    
    Layer = ds.GetRasterBand( int(band))
    
    XSize = Layer.XSize

    YSize = Layer.YSize
    
    return XSize, YSize
    

XSize,YSize = get_XSize_YSize(ds, 1)

print("XSize {0}, \t YSize {1} ".format(XSize,YSize))


def generate_coordinate_as_2D_array(ds, band=1):
    
    originX, pixelWidth, originY,  pixelHeight = get_GeoTransform_params(ds)
    
    
    XSize,YSize = get_XSize_YSize(ds, band)
    
    Xcoords = [((originX) + (pixelWidth)*x) for x in range(XSize)]
    
    Ycoords = [((originY) + (abs(pixelHeight))*x) for x in range(YSize)]
    
    return Xcoords, Ycoords

    
Coords = generate_coordinate_as_2D_array(ds, 1)


def get_distance(x, y):
    
    if x == None:
        x = 0
        
    vetor_result = x - y
    
    return np.linalg.norm( vetor_result)

def get_min(old,new):
    
    old_distance = get_distance(old, my_location)
     
    new_distance= get_distance(new, my_location)
    
    if old_distance  < new_distance:
        return old
    else:
        new
   

30.0 -30.0
XSize 7731, 	 YSize 7831 


In [111]:
my_location = np.array([562500, -2275850])

In [110]:
import pandas as pd

df = pd.DataFrame({'y':Coords[1]}, index=range(len(Coords[1])))

df['distance'] = np.abs(df['y'] - my_location[1])

df.sort_values('distance').head(4)

,y,distance
148,-2275845.0,5.0
147,-2275875.0,25.0
149,-2275815.0,35.0
146,-2275905.0,55.0


In [112]:
def get_idx_idy(coords=Coords, location_point=my_location):
    """Returns the respective nearest x index (idx) and the nearest y index (idy) 
        relative to the location_point_given
        
        params: 
            coords: the tuple containing the [x,y] coordinates of the image.
            
            location_point: a (x,y) tuple representing the coordinate of the respective Point
            
            returns: (idx, idy)  ::. the nearest index for x and the nearest index position for y relative to the array image.
        """
    idx = np.argmin(np.abs(coords[0] - location_point[0]))
    
    idy = np.argmin(np.abs(coords[1] - location_point[1]))
    
    return idx, idy

positioning_index = get_idx_idy()

In [ ]:
def get_dx_dy(ds):

    dx = ds.GetGeoTransform()[1]

    dy = ds.GetGeoTransform()[-1]
    return dx, dy

dx, dy = get_dx_dy(ds)

print(dx, dy)

def get_XSize_YSize(ds, band):
    
    Layer = ds.GetRasterBand( int(band))
    
    XSize = Layer.XSize

    YSize = Layer.YSize
    
    return XSize, YSize
    

XSize,YSize = get_XSize_YSize(ds, 1)

print("XSize {0}, \t YSize {1} ".format(XSize,YSize))


def generate_coordinate_as_2D_array(ds, band=1):
    
    originX, pixelWidth, originY,  pixelHeight = get_GeoTransform_params(ds)
    
    
    XSize,YSize = get_XSize_YSize(ds, band)
    
    Xcoords = [((originX) + (pixelWidth)*x) for x in range(XSize)]
   
    Ycoords = [((originY) + (abs(pixelHeight))*x) for x in range(YSize)]
    
    return Xcoords, Ycoords

    
Xcoords, Ycoords = generate_coordinate_as_2D_array(ds, 1)



Transverse_Mercator = ccrs.epsg(32622)

fig, ax = plt.subplots(subplot_kw={'projection':Transverse_Mercator})
Layer = ds.GetRasterBand(1)

Array = Layer.ReadAsArray()

ax.pcolormesh(Xcoords, Ycoords, Array, transform=Transverse_Mercator)

fig.show()

30.0 -30.0
XSize 7731, 	 YSize 7831 


In [40]:
Xsize = Layer.XSize

Ysize = Layer.YSize

print(Xsize, Ysize)

7731 7831


In [ ]:
Layer.